In [28]:
### CNN output may vary everytime due to reproducibility issues
### Using Kaggle GPU it takes only 22 minutes to run.
### CNN is the most feasible option for implementation as the data size grows.
### It does not require any kind of feature engineering - CNN does auto feature engineering.
### GPU makes crunching a big dataset easier and faster.
### CNN performance improves as the data increases - Also CNN has a good really CV.

# -> Our CNN can easily handle different features. So even if new meteorological features are added no worries !!!

In [1]:
import pandas as pd 
import numpy as np 
from tqdm.notebook import tqdm
import math
import gc
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split

import os
import shutil
import datetime
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

In [2]:
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.layers import *
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from joblib import Parallel, delayed
from functools import partial
import gc
from tensorflow.keras.layers import MaxPooling1D

In [1]:
# !mkdir input
# !cp /kaggle/input/zindi-airqo/* ./input

In [4]:
# Loading data
train = pd.read_csv('./input/Train.csv')
test = pd.read_csv('./input/Test.csv')

In [8]:
%%time
### Only takes 30s to preprocess the data
data = pd.concat([train,test]).reset_index(drop=True)
df = data[['ID', 'location', 'target']]
df = pd.concat([train,test]).reset_index(drop=True)
for c in tqdm_notebook(['temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press']):
    tmp = data[c].str.split(',', expand=True)
    tmp.columns = [c + '_' + str(x) for x in tmp.columns]
    df = pd.concat([df, tmp], axis=1)
obj_cols = [c for c in df.select_dtypes('object').columns if c not in ['ID', 'location']]
tmp = Parallel(n_jobs=4)(delayed(partial(pd.to_numeric, errors='coerce'))(df[c]) for c in obj_cols)
df = df.drop(obj_cols, axis=1)
df = pd.concat([df, pd.DataFrame(tmp).T], axis=1)
weather_cols = ['temp', 'precip', 'rel_humidity', 'wind_dir','wind_spd', 'atmos_press']

### Filling NaNs by interpolation.
for w in tqdm_notebook(weather_cols):
    selected_cols = [c for c in df.columns if w in c]
    df[selected_cols] = df[selected_cols].interpolate(limit_direction='both')
    ### Wind direction broken down into sine and cos components, and the original column is dropped
    if w == 'wind_dir':
        df[['wind_dir_cos_' + c.split('_')[-1] for c in selected_cols]] = df[selected_cols].apply(lambda x: np.cos(x))
        df[['wind_dir_sin_' + c.split('_')[-1] for c in selected_cols]] = df[selected_cols].apply(lambda x: np.sin(x))
        df = df.drop(selected_cols, axis = 1)    
print(df.shape)
features = [c for c in df.columns if c not in ['ID', 'location', 'target']]
print(df[features].isnull().sum().sum())
df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()



(20574, 857)
144018
CPU times: user 22 s, sys: 2.7 s, total: 24.7 s
Wall time: 33.5 s


,ID,location,target,temp,precip,rel_humidity,wind_spd,atmos_press,temp_0,temp_1,temp_2,temp_3,temp_4,temp_5,temp_6,temp_7,temp_8,temp_9,temp_10,temp_11,temp_12,temp_13,temp_14,temp_15,temp_16,temp_17,temp_18,temp_19,temp_20,temp_21,temp_22,temp_23,temp_24,temp_25,temp_26,temp_27,temp_28,temp_29,temp_30,temp_31,temp_32,temp_33,temp_34,temp_35,temp_36,temp_37,temp_38,temp_39,temp_40,temp_41,temp_42,temp_43,temp_44,temp_45,temp_46,temp_47,temp_48,temp_49,temp_50,temp_51,temp_52,temp_53,temp_54,temp_55,temp_56,temp_57,temp_58,temp_59,temp_60,temp_61,temp_62,temp_63,temp_64,temp_65,temp_66,temp_67,temp_68,temp_69,temp_70,temp_71,temp_72,temp_73,temp_74,temp_75,temp_76,temp_77,temp_78,temp_79,temp_80,temp_81,temp_82,temp_83,temp_84,temp_85,temp_86,temp_87,temp_88,temp_89,temp_90,temp_91,...,wind_dir_sin_21,wind_dir_sin_22,wind_dir_sin_23,wind_dir_sin_24,wind_dir_sin_25,wind_dir_sin_26,wind_dir_sin_27,wind_dir_sin_28,wind_dir_sin_29,wind_dir_sin_30,wind_dir_sin_31,wind_dir_sin_32,wind_dir_sin_33,wind_dir_sin_34,wind_dir_sin_35,wind_dir_sin_36,wind_dir_sin_37,wind_dir_sin_38,wind_dir_sin_39,wind_dir_sin_40,wind_dir_sin_41,wind_dir_sin_42,wind_dir_sin_43,wind_dir_sin_44,wind_dir_sin_45,wind_dir_sin_46,wind_dir_sin_47,wind_dir_sin_48,wind_dir_sin_49,wind_dir_sin_50,wind_dir_sin_51,wind_dir_sin_52,wind_dir_sin_53,wind_dir_sin_54,wind_dir_sin_55,wind_dir_sin_56,wind_dir_sin_57,wind_dir_sin_58,wind_dir_sin_59,wind_dir_sin_60,wind_dir_sin_61,wind_dir_sin_62,wind_dir_sin_63,wind_dir_sin_64,wind_dir_sin_65,wind_dir_sin_66,wind_dir_sin_67,wind_dir_sin_68,wind_dir_sin_69,wind_dir_sin_70,wind_dir_sin_71,wind_dir_sin_72,wind_dir_sin_73,wind_dir_sin_74,wind_dir_sin_75,wind_dir_sin_76,wind_dir_sin_77,wind_dir_sin_78,wind_dir_sin_79,wind_dir_sin_80,wind_dir_sin_81,wind_dir_sin_82,wind_dir_sin_83,wind_dir_sin_84,wind_dir_sin_85,wind_dir_sin_86,wind_dir_sin_87,wind_dir_sin_88,wind_dir_sin_89,wind_dir_sin_90,wind_dir_sin_91,wind_dir_sin_92,wind_dir_sin_93,wind_dir_sin_94,wind_dir_sin_95,wind_dir_sin_96,wind_dir_sin_97,wind_dir_sin_98,wind_dir_sin_99,wind_dir_sin_100,wind_dir_sin_101,wind_dir_sin_102,wind_dir_sin_103,wind_dir_sin_104,wind_dir_sin_105,wind_dir_sin_106,wind_dir_sin_107,wind_dir_sin_108,wind_dir_sin_109,wind_dir_sin_110,wind_dir_sin_111,wind_dir_sin_112,wind_dir_sin_113,wind_dir_sin_114,wind_dir_sin_115,wind_dir_sin_116,wind_dir_sin_117,wind_dir_sin_118,wind_dir_sin_119,wind_dir_sin_120
0,ID_train_0,C,45.126304,NaN,NaN,NaN,NaN,NaN,22.533333,21.716667,20.833333,20.983333,20.875000,20.141667,19.375000,18.791667,18.775000,18.641667,18.558333,18.533333,19.608333,21.916667,24.716667,26.658333,29.175000,30.700000,31.433333,32.333333,33.008333,33.391667,33.616667,31.091667,27.550000,26.658333,25.675000,26.425000,23.783333,22.416667,22.041667,21.008333,21.475000,21.825000,21.158333,22.308333,23.116667,22.333333,23.850000,23.825000,24.983333,28.050000,30.841667,31.991667,32.041667,32.000000,31.666667,29.158333,25.866667,24.300000,23.683333,23.758333,22.991667,21.891667,21.158333,20.416667,19.533333,19.125000,18.766667,17.983333,18.458333,21.933333,23.791667,25.558333,27.758333,29.658333,31.391667,31.883333,32.358333,32.708333,31.983333,30.850000,28.800,26.491667,24.866667,24.366667,23.025000,22.325000,21.650000,20.750000,20.475000,19.641667,19.516667,19.575000,20.000000,23.358333,25.608333,26.883333,26.358333,27.325000,29.008333,28.433333,...,-0.141620,0.677692,-0.691042,-0.570434,-0.621553,0.054916,-0.068411,-0.427471,-0.748119,0.795807,0.997073,-0.105681,-0.170296,0.850561,0.971100,-0.002620,-0.897881,0.855572,0.562276,0.606049,0.902810,-0.853162,-0.186615,-0.032798,0.937824,-0.317277,-0.451505,-0.797416,-0.159037,-0.842704,0.590000,0.249485,0.643785,0.369674,-0.910274,-0.390897,0.531411,-0.117671,0.682072,0.997411,-0.445040,0.641554,0.917998,0.647900,0.843400,0.831011,-0.780758,-0.402907,-0.970055,-0.775132,-0.737531,-0.877086,0.706791,-0.991253,0.870372,0.235040,-0.399780,-0.890181,-0.429068,0.536505,0.849812,-0.921784,0.890076,0.325722,-0.9689

In [9]:
### Encoding each location by the mean of its target
features = ["temp","precip","rel_humidity","wind_spd","atmos_press", "wind_dir_cos", "wind_dir_sin"]
df['location'] = df['location'].map(df.groupby('location')['target'].mean())

In [10]:
### Creating more features

new_features = []
for f in tqdm_notebook(features):
    new_features.append(f'{f}_24hrs_ratio')
    new_features.append(f'{f}_6hrs_ratio')
    for hour, f_col in enumerate([f"f_{i}" for i in range(121)]):
        
        ### Ratio of current hour and 24 hours before
        if hour - 24 < 0:
            df[f'{f}_24hrs_ratio_{hour}'] = df[f'{f}_{hour}'] /(1e-5 + df[f'{f}_0'])
        else:
            df[f'{f}_24hrs_ratio_{hour}'] = df[f'{f}_{hour}'] /(1e-5 + df[f'{f}_{hour-24}'])

            
        ### Ratio of current hour and 6 hours before
        if hour - 6 < 0:
            df[f'{f}_6hrs_ratio_{hour}'] = df[f'{f}_{hour}'] /(1e-5 + df[f'{f}_0'])
        else:
            df[f'{f}_6hrs_ratio_{hour}'] = df[f'{f}_{hour}'] /(1e-5 + df[f'{f}_{hour-6}'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [12]:
features = features + new_features
features

['temp',
 'precip',
 'rel_humidity',
 'wind_spd',
 'atmos_press',
 'wind_dir_cos',
 'wind_dir_sin',
 'temp_24hrs_ratio',
 'temp_6hrs_ratio',
 'precip_24hrs_ratio',
 'precip_6hrs_ratio',
 'rel_humidity_24hrs_ratio',
 'rel_humidity_6hrs_ratio',
 'wind_spd_24hrs_ratio',
 'wind_spd_6hrs_ratio',
 'atmos_press_24hrs_ratio',
 'atmos_press_6hrs_ratio',
 'wind_dir_cos_24hrs_ratio',
 'wind_dir_cos_6hrs_ratio',
 'wind_dir_sin_24hrs_ratio',
 'wind_dir_sin_6hrs_ratio']

In [13]:
train=df[df.target.notnull()].reset_index(drop=True)
test=df[df.target.isna()].reset_index(drop=True)

In [14]:
def get_sample(x):
    '''
    Convert a train row into a sample suitable for CNN
    x: row of train dataset
    '''
    sub_sample = np.zeros((len(features)+1, 121))
    for i, f in enumerate(features + ["location"]):
        if f == "location":
            sub_sample[i] = x["location"] * np.ones(121)
        else:
            cols_f = [f"{f}_{i}" for i in range(121)]
            vals = x[cols_f].values.astype(float)
            sub_sample[i] = vals
    return np.array(sub_sample).astype('float')

In [15]:
train.shape

(15539, 2551)

In [16]:
%%time
train_samples = Parallel(n_jobs=4)(delayed(get_sample)(row[1]) for row in train.iterrows())
test_samples = Parallel(n_jobs=4)(delayed(get_sample)(row[1]) for row in test.iterrows())

CPU times: user 1min 43s, sys: 3.45 s, total: 1min 47s
Wall time: 6min 39s


In [17]:
X_test = np.array(test_samples)
X_test = np.einsum('ikj->ijk', X_test)
X_train = np.array(train_samples)
X_train = np.einsum('ikj->ijk', X_train)
X_train.shape, X_test.shape

((15539, 121, 22), (5035, 121, 22))

In [18]:
X_all = np.concatenate([X_train, X_test])

my_mean = X_all.mean(axis=(0, 1))
my_std = X_all.astype(float).std(axis=(0, 1))

X_train[:, :] -= my_mean
X_test [:, :] -= my_mean

X_train[:, :] /= my_std
X_test [:, :] /= my_std

print(f"Train std = {X_train.std(): .3f}, Test std = {X_test.std(): .3f}, \nTrain mean = {X_train.mean(): .3f}, Test mean = {X_test.mean(): .3f}")

Train std =  1.020, Test std =  0.935, 
Train mean = -0.001, Test mean =  0.002


In [19]:
y_train = train["target"].values
y_train.shape

(15539,)

In [20]:

N_FEATS = X_train.shape[2]


def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

def make_model():
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64, kernel_size=(3,), strides=1, activation='relu', input_shape=(121, N_FEATS)))
    model.add(Conv1D(filters=64, kernel_size=(5 ,), strides=1, activation='relu'))
    
    model.add(Conv1D(filters=128, kernel_size=(7, ), strides=1, activation='relu'))
    model.add(Conv1D(filters=128, kernel_size=(11, ), strides=1, activation='relu'))
    model.add(Conv1D(filters=256, kernel_size=(17, ), strides=1, activation='relu'))
    model.add(MaxPooling1D(4))
    

    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Dense(32, activation='relu', name='output_cnn_simple'))
    model.add(Dense(1))

    model.compile(loss=root_mean_squared_error, optimizer=Adam(lr = 5e-4))
    
    return model

m = make_model()
m.build((None, 121, N_FEATS))
m.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo multiple                  88        
_________________________________________________________________
conv1d (Conv1D)              multiple                  4288      
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  20544     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  57472     
_________________________________________________________________
conv1d_3 (Conv1D)            multiple                  180352    
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  557312    
_________________________________________________________________
max_pooling1d (MaxPooling1D) multiple                  0

In [21]:
errcb2 = list()
y_pred_test = list()


for my_round in range(1):
    i = 0
    print(f"\n_____________________Round n-{my_round + 1}\n")
    
    fold = KFold(n_splits=10, shuffle=True, random_state=my_round)
    
    for train_index, test_index in fold.split(X_train, y_train):
        i += 1
        print(f"\nFold n-{i}\n")

        X_entr, X_val = X_train[train_index], X_train[test_index]
        y_entr, y_val = y_train[train_index], y_train[test_index]

        es = tf.keras.callbacks.EarlyStopping(patience=10, 
                                           verbose=1, 
                                           restore_best_weights=True)

        m = make_model()

        m.fit(X_entr,
                y_entr,
                epochs=150,
                verbose=0,
                batch_size=128,
                validation_data=(X_val, y_val),
                callbacks=[es],
             )

        # val score
        preds = m.predict(X_val)[:, 0]
        rmse_val = mean_squared_error(y_val, preds) ** .5
        print(f"Erreur = {rmse_val:.2f}\n\n")
        errcb2.append(rmse_val)

        # prediction
        p2 = m.predict(X_test)[:, 0]
        p2 = np.clip(p2, train.target.min(), train.target.max())

        y_pred_test.append(p2)

print(f"Total error = {np.mean(errcb2): .3f}")


_____________________Round n-1


Fold n-1

Restoring model weights from the end of the best epoch.
Epoch 00060: early stopping
Erreur = 23.69



Fold n-2

Restoring model weights from the end of the best epoch.
Epoch 00050: early stopping
Erreur = 23.92



Fold n-3

Restoring model weights from the end of the best epoch.
Epoch 00069: early stopping
Erreur = 20.91



Fold n-4

Restoring model weights from the end of the best epoch.
Epoch 00039: early stopping
Erreur = 22.98



Fold n-5

Restoring model weights from the end of the best epoch.
Epoch 00055: early stopping
Erreur = 21.24



Fold n-6

Restoring model weights from the end of the best epoch.
Epoch 00046: early stopping
Erreur = 24.60



Fold n-7

Restoring model weights from the end of the best epoch.
Epoch 00054: early stopping
Erreur = 24.30



Fold n-8

Restoring model weights from the end of the best epoch.
Epoch 00045: early stopping
Erreur = 24.49



Fold n-9

Restoring model weights from the end of the best epoch.
Epoc

In [22]:
d = {'ID': test["ID"], 'target': np.mean(y_pred_test, axis=0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]
sub['target'] = sub['target'].clip(train.target.min(), train.target.max())

In [25]:
sub.head(10)

,ID,target
0,ID_test_0,146.448013
1,ID_test_1,78.831528
2,ID_test_10,33.348366
3,ID_test_100,55.782494
4,ID_test_1000,88.687622
5,ID_test_1001,30.877085
6,ID_test_1002,75.518860
7,ID_test_1003,37.986477
8,ID_test_1004,34.908413
9,ID_test_1005,52.110146


In [29]:
np.mean(errcb2)

23.151077529050763

In [32]:
sub.to_csv(f"cnn_preds.csv", index=False)

In [33]:
SUB_FILE_NAME = f"cnn_preds.csv"
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)
create_download_link(filename = SUB_FILE_NAME)